In [ ]:
"""
Copyright (c) 2025 viewstar000

This software is released under the MIT License.
https://opensource.org/licenses/MIT
"""

%load_ext jupyter_agent.bot_magics
%config BotMagics.default_api_url = 'http://127.0.0.1:1234/v1'
%config BotMagics.default_api_key = 'API_KEY'
%config BotMagics.default_model_name = 'qwen3-30b-a3b' 
%config BotMagics.coding_model_name = 'devstral-small-2505-mlx'
%config BotMagics.support_save_meta = True  # Need jupyter-agent-extension installed
%config BotMagics.support_set_cell_content = True # Need jupyter-agent-extension installed

# 默认情况下，code类型的单元格会加入代码生成上下中，markdown类型的的单元格会加入到任务规划上下中。
# 添加"CTX_CODE"标记到cell的tags中，表示当前单元会加入到代码生成上下文中。
# 添加"CTX_TASK"标记到cell的tags中，表示当前单元会加入到任务规划上下文中。
# 添加"CTX_EXCLUDE"标记到cell的tags中，表示当前单元会排除在LLM会话上下文之外。

In [ ]:
%%bot -P # 定义全局目标并执行全局规则

# 全局目标

你是一个专业的数据清洗助手。你的主要任务是帮助用户分析原始数据，识别数据质量问题，并最终生成可复用的 Python 数据清洗脚本与数据加载函数或类。你具备以下能力：

### 🎯 核心功能与步骤

- 接收数据：接收用户提供的数据存储路径，通常是一个文件夹或压缩文件包
- 分析目录结构：分析路径下的文件目录结构、文件格式等，输出文件目录结构描述、文件类型统计，通过推理分析总结出路径pattern等
- 原始数据schema推断：逐一加载所有的样本文件，输出其字段列表，合并字段列表并推断出原始数据的schema
- 预览数据：加载并预览数据，输出样本数据、字段信息、统计描述等
  - 需要加载并合全部数据后再进行统计
  - 对于数值类字段，输出其统计描述，如均值、方差、最大值、最小值、缺失比例等
  - 对于枚举类字段，统计其所有权举值的数量分布（可以考虑只输出top100)
  - 注意：**该步骤不需要对数据Schema进行检查**，只需要输出处理建议供后续步骤参考
- 生成清洗规则：结合用户提供的目标数据Schema，检查并输出缺失值、异常值、重复记录、格式错误、Schema不一致等问题列表以及相应的清先策略建议
  - 对于字段名不一致的情况，需要跟据数据的内容推断给出合理的映射的规则
  - 对于枚举值不一致的情况，需要跟据数据的内容推断给出合理的映射的规则
  - 对数值量纲不一致的情况，需要跟据数据的内容推断给出合理的映射的规则
  - 该步骤不需要进行实际的清洗操作，只需要输出处理建议供后续步骤参考
- 生成数据加载器：结合数据分析与诊断结果及用户提供的目标数据Schema，根据用户选择或默认规则对生成数据清洗与加载器代码
   - 将清洗逻辑封装为函数或类，形成“数据加载器”模块
   - 运行清洗代码，并对清洗后的结果进行预览确认

### ⚙️ 技术栈要求

- 使用 Python（Pandas 为主）；
- 输出结构化数据（如 DataFrame 预览）；
- 所有操作需保证数据完整性，不修改原始数据源；

In [ ]:
### 数据集的根目录

DATASET_ROOT = "var/sapimouse"

In [ ]:
### 定义目标数据Schema

from typing import TypedDict
from enum import Enum


class MouseButton(Enum):

    LEFT = 1
    RIGHT = 2
    MIDDLE = 3
    WHEEL_UP = 4
    WHEEL_DOWN = 5
    WHEEL_LEFT = 6
    WHEEL_RIGHT = 7
    NONE = 0
    UNKNOWN = -1


class MouseEvent(Enum):

    PRESS = 1
    RELEASE = 2
    MOVE = 3
    DRAG = 4
    WHEEL = 5
    NONE = 0
    UNKNOWN = -1


class MouseTraceSchema(TypedDict):
    source: str  # 数据源标识，尽量简短明确，如 "sapimouse"
    user_id: str  # 用户唯一标识，可选
    session_id: str  # 会话唯一标识，可选
    timestamp: float  # 时间戳，单位为秒
    button: MouseButton  # 鼠标按键，使用枚举值
    event: MouseEvent  # 鼠标事件类型，使用枚举值
    x: float  # 鼠标X坐标
    y: float  # 鼠标Y坐标

In [ ]:
%%bot # 开始规划、生成、执行、调试子任务代码。可重复追加当前命令，直到全局任务完成。

# Execute this cell to generate the next task
# 2025-07-01T11:53:32.650974，不可省略，但可以用任务随机字符串代替
# 特别注意：执行前必须确保当前notebook已保存